<a href="https://colab.research.google.com/github/moey920/NLP/blob/master/%EB%A0%88%EB%B2%A4%EC%8A%88%ED%83%80%EC%9D%B8_%EA%B1%B0%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install soynlp

# -*- coding: utf-8 -*-
import re
import sys
from os import listdir
from os.path import isfile, join, basename

from soynlp.hangle import levenshtein
from soynlp.hangle import jamo_levenshtein

from konlpy.tag import Kkma, Okt
from konlpy.utils import pprint
# from sklearn.feature_extraction.text import TfidfVectorizer

def edit_distance(s1, s2):
    s1 = re.split(' |\n|\t', s1)
    s1 = filter(None, s1)
    s2 = re.split(' |\n|\t', s2)
    s2 = filter(None, s2)
    s1 = ' '.join(s1)
    s2 = ' '.join(s2)
    #    print s1
    #    print s2
    l1 = len(s1)
    l2 = len(s2)
    if l2 == 0:
        return l1
    # previous_row = xrange(l2 + 1)
    previous_row = range(l2 + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        flag = 0
        for j, c2 in enumerate(s2):
            insertions = previous_row[
                             j + 1] + 1  # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1  # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return (float(previous_row[-1]), float(l2))


def ocr_ans(user, ground):
    kkma = Kkma()
    okt = Okt()

    userfiles = [f for f in listdir(user) if isfile(join(user, f))]
    userfiles.sort()
    groundfiles = [f for f in listdir(ground) if isfile(join(ground, f))]
    s = 0.0
    count = 0
    outputfile = open('output.txt', 'w+')

    for f in userfiles:
        # mydoclist_kkma = []
        # okt_ground = []
        # okt_user = []

        f2 = (f.split('_'))[:-1]
        # f2 = (f.split('_'))[:1]
        # f2 = basename(f)
        # f2 = f
        # filename = ground + '/' + '_'.join(f2) + '.txt'
        if len(f2) == 0:
            f2 = f.split('.')[:-1]
        if len(f2) == 0:
            # print("debug ----- can not get user file name body >> continue")
            continue
        # ground_filename = join(ground, '_'.join(f2) + '_hand.txt')
        ground_filename = join(ground, f2[0] + '_hand.txt')
        '''
        filenames = []
        filenames.append(join(ground, '_'.join(f2) + '.txt'))
        filenames.append(join(ground, '_'.join(f2) + '_masked.txt'))
        filenames.append(join(ground, '_'.join(f2) + '_box_transform.txt'))
        filenames.append(join(ground, '_'.join(f2) + '_vs_transform.txt'))
        '''
        # print 'user : ', user + '/' + f
        # print("user:", user + '/' + f)
        # userfile = open(user + '/' + f, 'r')
        # userfile = open(join(user, f), 'rt', encoding='utf-8')
        user_filename = join(user, f)
        try:
            with open(user_filename, 'rt', encoding='utf-8') as userfile:
                # print("user file:", user_filename)
                user_text = userfile.read()
        except IOError as exc:
            '''
            tb = sys.exc_info()[-1]
            lineno = tb.tb_lineno
            src = tb.tb_frame.f_code.co_filename
            print('{} at {} line {}.'.format(exc.strerror, src, lineno))
            '''
            continue
        '''
        try:
            user_text = user_text.decode('utf-8')
        except UnicodeDecodeError:
            try:
                user_text = user_text.decode('utf-16')
            except UnicodeDecodeError:
                continue
        '''
        # print "ground : ", filename
        try:
            with open(ground_filename, 'rt', encoding='utf-8') as groundfile:
                # print("ground file:", ground_filename)
                ground_text = groundfile.read()
            '''
            try:
                ground_text = ground_text.decode('utf-8')
            except UnicodeDecodeError:
                try:
                    ground_text = ground_text.decode('utf-16')
                except UnicodeDecodeError:
                    continue
            '''
        except IOError as exc:
            '''
            tb = sys.exc_info()[-1]
            lineno = tb.tb_lineno
            src = tb.tb_frame.f_code.co_filename
            print('{} at {} line {}.'.format(exc.strerror, src, lineno))
            '''
            continue

        # 자기 자신을 넣어보면 e[0]/e[1]은 0.0이 나온다.
        # e = edit_distance(ground_text, ground_text)
        # print("debug ---- dist ground_text/ground_text", e[0]/e[1])
        e = edit_distance(user_text, ground_text)
        # print(count, basename(user_filename), basename(ground_filename), e[0] / e[1])

        # soynlp 루벤스타인+자모 루벤스타인 조사
        l = levenshtein(user_text, ground_text)
        jl = jamo_levenshtein(user_text, ground_text)

        # konlpy 를 사용해서 명사만 구하고 이를 다시 soynlp 루벤스타인+자모 루벤스타인 조사
        okt_user_nouns = ' '.join(okt.nouns(user_text))
        okt_ground_nouns = ' '.join(okt.nouns(ground_text))
        # okt_user.append(okt_user_nouns)
        # okt_ground.append(okt_ground_nouns)
        lo = levenshtein(okt_user_nouns, okt_ground_nouns)
        jlo = jamo_levenshtein(okt_user_nouns, okt_ground_nouns)

        outputstring = (
                str(count) + '\t' +
                basename(user_filename) + '\t' + basename(ground_filename) + '\t' +
                str(float(e[0]) / float(e[1]) * 100) + '\t' +
                str(l) + '\t' + str(jl) + '\t' +
                str(lo) + '\t' + str(jlo)
        )
        print(outputstring)
        outputfile.write(outputstring+'\n')
        count += 1
        '''
        s += e[0] / e[1]
        outputstring = '_'.join(f2) + '\t' + str(e[1]) + '\t' + str(e[0]) + '\t' + str(
            float(e[0]) / float(e[1]) * 100) + '\n'
        outputfile.write(outputstring)
        count += 1
        print(s, count)
        print(s / float(count))
        '''

ocr_ans(sys.argv[1], sys.argv[2])
